In [1]:
import pandas as pd
import os
import json
import pymongo

In [49]:
# EXTRACT

# Incarceration Rates
incar_df = pd.read_html('https://commons.wikimedia.org/wiki/File:U.S._incarceration_rates_1925_onwards.png')[1]

# Birth Rates
births_total_path = os.path.join('.','dirty_csv',"TotalBirthsandFertilityRate1909-2015.csv")
births_unmarried_path = os.path.join('.','dirty_csv',"UnmarriedBirthsbyAgeGroup1940-2015.csv")
births_total_df = pd.read_csv(births_total_path)
births_unmarried_df = pd.read_csv(births_unmarried_path)

# Property Crime Rates
prop_crime_path = os.path.join('.','dirty_csv',"PropertyCrimeRate1960-2014.csv")
prop_crime_df = pd.read_csv(prop_crime_path)

In [50]:
# TRANSFORM

# Incarceration Rates
incar_df.columns = incar_df.iloc[0]
incar_df = incar_df.drop([0])

incar_df_path = os.path.join('.','cleaned_json','incarceration_rate.json')
incar_df.to_json(f'{incar_df_path}')
incar_jdf = open(f'{incar_df_path}').read()
incar_data = json.loads(incar_jdf) 

# Birth Rates
births_total = births_total_df['Birth Number']
births_unmarried_df.insert(3, "Total Population", "")
births_unmarried_df.insert(4, "Total Births", "")
births_unmarried_df.insert(5, "% of Total Births", "")

births_teen_df = pd.DataFrame(columns = ['Year','Age Group', 'Number of Births', 'Total Births'])

for index, row in births_unmarried_df.iterrows():
    year_row = births_total_df[births_total_df['Year'] == row['Year']]
    pop_size = int(year_row['Birth Number'] / (year_row['Crude Birth Rate']/1000) )
    row['Total Births'] = int(year_row['Birth Number'])
    row['% of Total Births'] = row['Number of Births'] * 100 / row['Total Births']
    births_unmarried_df.iloc[index] = row

    
for index, row in births_unmarried_df.iterrows():
    if births_unmarried_df.iloc[index]['Age Group'] == 'Less than 15 years' or \
    births_unmarried_df.iloc[index]['Age Group'] == '15-19 years' or \
    births_unmarried_df.iloc[index]['Age Group'] == '20-24 years':
        births_teen_df = births_teen_df.append(births_unmarried_df.iloc[index])
        
percent_births = births_teen_df.groupby(by=['Year']).sum()
percent_births = percent_births.drop([1940, 2015])

percent_df_path = os.path.join('.','cleaned_json','births_unmarried_under_25.json')
percent_births.to_json (f'{percent_df_path}')
births_jdf = open(f'{percent_df_path}').read()
births_data = json.loads(births_jdf)

# Property Crime Rates
prop_crime_df = prop_crime_df.drop('Unnamed: 6', axis = 1)
prop_crime_df = prop_crime_df.dropna()

prop_crime_df.set_index('Year', inplace = True)
prop_crime_df['Property crime rate'] = prop_crime_df['Property crime rate'].str.lstrip()
prop_crime_df = prop_crime_df.astype({'Property crime rate': 'float'})

prop_crime_df_path = os.path.join('.','cleaned_json','property_crime_rate.json')
prop_crime_df.to_json (f'{prop_crime_df_path}')
prop_crime_jdf = open(f'{prop_crime_df_path}').read()
prop_crime_data = json.loads(prop_crime_jdf)

In [57]:
# LOAD

connection_string = 'mongodb://localhost:27017'
client = pymongo.MongoClient(connection_string)
db = client.etl_DB
iR = db.incarceration_Rates
bR = db.birth_Rates
cR = db.crime_Rates

# Incarceration Rates
incar_df.index = incar_df.index.map(str)
incar_dict = incar_df.T.to_dict()

iR.insert_one(incar_dict)

# # Birth Rates
percent_births.index = percent_births.index.map(str)
percent_births_dict = percent_births.T.to_dict()

bR.insert_one(percent_births_dict)

# Property Crime Rates
prop_crime_df.index = prop_crime_df.index.map(str)
prop_crime_dict = prop_crime_df.T.to_dict()

cR.insert_one(prop_crime_dict)